**16S V1-V9 diff kmers**

In [1]:
import numpy as np
import pandas as pd
import os
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

from os import listdir
from pandas import read_csv, DataFrame
from tqdm import tqdm
from subprocess import call
from Bio.SeqIO import parse
from skbio.stats.composition import clr 


In [ ]:
# one-time install
pip install -U kaleido

In [2]:
def create_fasta(output, mearged_pike_out, dbpath):
    # Create fasta file 
    consensus = {}
    cons_conter = 0
    
    with open(f'{output}/all_consensus.fasta', 'w') as opn_fasta:
        for cons in mearged_pike_out.index:
    
            opn_fasta.write(f'>{cons_conter}\n{cons}\n')
            consensus[cons_conter] = cons
            cons_conter += 1
    
    return consensus
    
def run_blast(base, path):
    
    call(f'makeblastdb -in {base} -dbtype nucl', shell=True)
    call(f'blastn -num_threads 60  -outfmt "7 qseqid sseqid pident evalue qcovs bitscore" -query {path}/all_consensus.fasta  -db {base} -out {path}/blast_results.txt', shell=True)
 #   pass
def decode_tax(base) -> dict:
    
    # DB decoder 
    # Use db header format: Kingdom    Phylum    Class    Order    Family    Genus    Species
    
    base = parse(base, 'fasta')
    taxonomy_linage = ['Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
    tax_decoder = {}
    
    for line in tqdm(base):
        
        tax_decoder[line.id] = {}
        linage = line.description.split(';')
        linage[0] = linage[0].split()[1]
    
        for i in range(len(taxonomy_linage)):
            try:
                if taxonomy_linage[i] != 'Species':
                
                    tax_decoder[line.id][taxonomy_linage[i]] = linage[i]
                
                else:
                    #print(linage[i].split())
                    tax_decoder[line.id][taxonomy_linage[i]] = ' '.join(linage[i].split()[:2])
                  
            except:
                
                tax_decoder[line.id][taxonomy_linage[i]] = 'NA'
    
    return tax_decoder

def parse_blast(path, 
                base, 
                data_tax, 
                consensus, 
                identity_filter, 
                cov_lim, 
                evalue_filter):
    
    # parser of blast table
    
    blast_header = ['qseqid',
                    'sseqid', 
                    'pident',
                    'evalue',
                    'qcovs', 
                    'bitscore']
    
    blasting_results = {}
    opn_blast = read_csv(f'{path}/blast_results.txt', sep='\t', comment='#', header=None, names=blast_header)
    
    for i in tqdm(opn_blast['qseqid'].unique()):
        
        blast_subset = opn_blast[opn_blast["qseqid"] == i]
        blast_subset = blast_subset[blast_subset['pident'] >= identity_filter]
        blast_subset = blast_subset[blast_subset['evalue'] <= evalue_filter]
        blast_subset = blast_subset[blast_subset['qcovs'] >= cov_lim]

        blast_subset = blast_subset.sort_values(by='evalue')
        blast_subset = blast_subset.sort_values(by='pident')[::-1]

        if len(blast_subset['sseqid'].values) == 0:
            continue
            
        subject = blast_subset['sseqid'].values[0]
        blasting_results[consensus[i]] = data_tax[subject]
        
    blasting_results_df = DataFrame(blasting_results).T
    
    return blasting_results_df
    
def processing_data_tax(data_tax):

    data_tax_df = DataFrame(data_tax).T.fillna(0)
    # Add pseudocunt
    # data_tax_df = data_tax_df + 1
    data_tax_df = data_tax_df.assign(m=data_tax_df.mean(axis=1)).sort_values('m').drop('m', axis=1)[np.sort(data_tax_df.columns)]

    return  data_tax_df
    
def get_taxonomy(data_tax, 
                 blasting_results_df, 
                 mearged_pike_out, 
                 tax_level='OTU'):
    
    data_tax = {}
    avs = np.intersect1d(blasting_results_df.index, mearged_pike_out.index)
    count = 1
    OTU_decoder  = {'Seq': [], 'OTU_name' : []}
    
    for av in tqdm(avs):

        if tax_level == 'OTU':
        
            tax = f'OTU_{count}_{blasting_results_df["Species"][av]}'
        else:    
            tax = blasting_results_df[tax_level][av]
        count += 1
        OTU_decoder['Seq'].append(av)
        OTU_decoder['OTU_name'].append(tax)
        if tax == 'nan':
            
            tax = 'No Fungi'

        if tax not in data_tax.keys():
    
            data_tax[tax] = {col: 0 for col in mearged_pike_out.columns} 
        
        for col in mearged_pike_out.columns:
           
            data_tax[tax][col] += mearged_pike_out[col][av]
    
    data_tax_df = processing_data_tax(data_tax)
    
    return data_tax_df, OTU_decoder
    

def filter_data(output, 
                dbpath,
                mearged_pike_out,
                taxonomy_level, 
                identity_filter=95, 
                cov_lim=60, 
                evalue_filter=1e-05):

    # Creating output directory
    try:
        
        os.mkdir(output)
        
    except FileExistsError:
        
        print('The output directory already exists!')
        
    consensus = create_fasta(output, mearged_pike_out, dbpath)
    run_blast(dbpath, output)
    data_tax = decode_tax(dbpath)
    blasting_results_df = parse_blast(output, 
                                      dbpath, 
                                      data_tax, 
                                      consensus, 
                                      identity_filter, 
                                      cov_lim, 
                                      evalue_filter)

  #  mearged_pike_out = filter_av(mearged_pike_out, prevalence, detection, slice)
    data_tax_df, OTU_decoder = get_taxonomy(data_tax, 
                                            blasting_results_df, 
                                            mearged_pike_out,
                                            taxonomy_level)

    data_tax_df = data_tax_df[mearged_pike_out.columns]
    for col in data_tax_df.columns:
        
        data_tax_df[col] = data_tax_df[col] / np.sum(data_tax_df[col].values)
    
    data_tax_df = data_tax_df.fillna(0)[mearged_pike_out.columns]   
    data_tax_df = data_tax_df.assign(m=data_tax_df.mean(axis=1)).sort_values('m').drop('m', axis=1)


    return data_tax_df, data_tax, blasting_results_df, DataFrame(OTU_decoder)

In [4]:
import random
def get_color(obj_dict):
    
    color = ''
    
    while color not in obj_dict.values() and color == '':
        
        color = "#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
    
    return color

MERGING ALL TABLES INTO ONE

In [7]:
mearged_otu_table = []

#for kmer_type in ['_1_kmer_size', '_2_kmer_size', '_3_kmer_size', '_4_kmer_size','_5_kmer_size', '_6_kmer_size','_7_kmer_size']:
for kmer_type in ['_k_3', '_k_4','_k_5', '_k_6','_k_7']:    
    for sample in listdir(f'/mnt/AsusShareI2/RUNS/runs-sonec/pool_diff_kmers/16S/pool_V3_V4{kmer_type}/results/'):
        opn_res = read_csv(f'/mnt/AsusShareI2/RUNS/runs-sonec/pool_diff_kmers/16S/pool_V3_V4{kmer_type}/results/{sample}/results.tsv', sep='\t', index_col=0)
        if 'Count' in opn_res.columns:
            count = 0
            mearged_otu_table.append(DataFrame(data=opn_res['Count'].tolist(), index=opn_res.index, columns=[sample + kmer_type]))
            with open(f'/mnt/AsusShareI2/RUNS/runs-sonec/pool_diff_kmers/16S/pool_V3_V4{kmer_type}/{sample}.fasta', 'w') as opn_fasta:
                for line in opn_res.index:
                    opn_fasta.write(f'>{count}_{opn_res["Count"][line]}\n{line}\n')
                    count += 1
        else:
            print(f'Столбец "Count" отсутствует в результате для образца {sample + kmer_type}')

mearged_otu_table = pd.concat(mearged_otu_table, axis=1).fillna(0)
mearged_otu_table = mearged_otu_table.reindex(sorted(mearged_otu_table.columns), axis=1)
mearged_otu_table.to_csv('pool_kmers_3_7_V3_V4_merged_otu_table_all_samples.csv')

In [ ]:
mearged_otu_table.head(10)

**WORKING WITH SILVA AND BLAST**

In [11]:
output = f'/mnt/AsusShareI2/RUNS/runs-sonec/pool_diff_kmers/16S/pool_V1_V9{kmer_type}/TAXONOMY'
dbpath = '/mnt/AsusShareI2/RUNS/runs-sonec/SILVA_138.1_SSURef_NR99_tax_silva.fasta'
#taxonomy_level = 'Genus'
taxonomy_level = 'Species'
#taxonomy_level = 'OTU'

data_tax_df, data_tax, blasting_results_df, OTU_decoder = filter_data(output, 
                                                                     dbpath,
                                                                     mearged_otu_table,
                                                                     taxonomy_level, 
                                                                     identity_filter=95, 
                                                                     cov_lim=60, 
                                                                     evalue_filter=1e-05)
data_tax_df.to_csv('pool_kmers_3_7_V3_V4_data_tax_df_species.csv', sep='\t')

The output directory already exists!


Building a new DB, current time: 08/09/2024 17:51:14
New DB name:   /mnt/AsusShareI2/RUNS/runs-sonec/SILVA_138.1_SSURef_NR99_tax_silva.fasta
New DB title:  /mnt/AsusShareI2/RUNS/runs-sonec/SILVA_138.1_SSURef_NR99_tax_silva.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /mnt/AsusShareI2/RUNS/runs-sonec/SILVA_138.1_SSURef_NR99_tax_silva.fasta
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 510508 sequences in 11.6813 seconds.




510508it [00:05, 88860.91it/s]
100%|█████████████████████████████████████████| 68/68 [00:00<00:00, 8314.75it/s]


In [62]:
data_tax_df.head(30)

,V1_V9_1_3_kmer_size,V1_V9_1_4_kmer_size,V1_V9_1_5_kmer_size,V1_V9_1_6_kmer_size,V1_V9_1_7_kmer_size,V1_V9_2_3_kmer_size,V1_V9_2_4_kmer_size,V1_V9_2_5_kmer_size,V1_V9_2_6_kmer_size,V1_V9_2_7_kmer_size,...,V1_V9_R2_3_kmer_size,V1_V9_R2_4_kmer_size,V1_V9_R2_5_kmer_size,V1_V9_R2_6_kmer_size,V1_V9_R2_7_kmer_size,V1_V9_R3_3_kmer_size,V1_V9_R3_4_kmer_size,V1_V9_R3_5_kmer_size,V1_V9_R3_6_kmer_size,V1_V9_R3_7_kmer_size
Ceuthophilus sp.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001210
Yersinia pestis,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002201,0.000000,0.000000
uncultured organism,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.007767,0.003540,0.000000,0.000000,0.000000,0.000000,0.000000
uncultured bacterium,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.002419,0.012719,0.010936,0.012793,0.000000,0.000000,0.017719,0.021677,0.019287
Staphylococcus epidermidis,0.003442,0.004659,0.005174,0.004716,0.004726,0.005200,0.005923,0.006401,0.006412,0.005933,...,0.006182,0.007066,0.006983,0.007022,0.006955,0.006835,0.007960,0.008328,0.006991,0.006857
Shigella boydii,0.000000,0.000000,0.033975,0.041562,0.041091,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.029603,0.000000,0.026595,0.000000,0.000000,0.000000,0.003779,0.003736,0.000000
Streptococcus thermophilus,0.000000,0.000000,0.000000,0.000000,0.000000,0.129145,0.128969,0.130889,0.128611,0.128807,...,0.000000,0.000000,0.000000,0.002734,0.002732,0.000000,0.000000,0.000000,0.000000,0.000000
Yersinia pseudotuberculosis,0.083614,0.082129,0.083233,0.084282,0.084465,0.094029,0.095933,0.097887,0.096183,0.096330,...,0.075247,0.077413,0.081489,0.076058,0.079240,0.079598,0.079896,0.072343,0.074908,0.078359
Escherichia coli,0.125040,0.127229,0.091055,0.084802,0.083023,0.115013,0.115535,0.105096,0.115298,0.114434,...,0.091795,0.057041,0.082861,0.059964,0.083960,0.079635,0.083392,0.081368,0.076684,0.082172
Streptococcus salivarius,0.126041,0.125020,0.124830,0.124366,0.124675,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.127451,0.124459,0.119708,0.118499,0.118425,0.120686,0.120179,0.115925,0.115710,0.115100


In [87]:
data_tax_no0_rows = (data_tax_df !=0).sum()
result = pd.DataFrame(data_tax_no0_rows).transpose()
result.head(10)
#result.index = ['OTU_count_nonzero']
#result.to_csv('OTU_count_nonzero.csv',sep='\t')

,V1_V9_1_3_kmer_size,V1_V9_1_4_kmer_size,V1_V9_1_5_kmer_size,V1_V9_1_6_kmer_size,V1_V9_1_7_kmer_size,V1_V9_2_3_kmer_size,V1_V9_2_4_kmer_size,V1_V9_2_5_kmer_size,V1_V9_2_6_kmer_size,V1_V9_2_7_kmer_size,...,V1_V9_R2_3_kmer_size,V1_V9_R2_4_kmer_size,V1_V9_R2_5_kmer_size,V1_V9_R2_6_kmer_size,V1_V9_R2_7_kmer_size,V1_V9_R3_3_kmer_size,V1_V9_R3_4_kmer_size,V1_V9_R3_5_kmer_size,V1_V9_R3_6_kmer_size,V1_V9_R3_7_kmer_size
0,9,15,14,11,11,9,12,13,10,11,...,9,17,16,18,16,9,12,19,19,18


In [88]:
result_long = result.melt(var_name='column', value_name='OTU_count_nonzero')
result_long

,column,OTU_count_nonzero
0,V1_V9_1_3_kmer_size,9
1,V1_V9_1_4_kmer_size,15
2,V1_V9_1_5_kmer_size,14
3,V1_V9_1_6_kmer_size,11
4,V1_V9_1_7_kmer_size,11
5,V1_V9_2_3_kmer_size,9
6,V1_V9_2_4_kmer_size,12
7,V1_V9_2_5_kmer_size,13
8,V1_V9_2_6_kmer_size,10
9,V1_V9_2_7_kmer_size,11


In [89]:
result_long['sample_name'] = result_long['column'].str.extract(r'(V1_V9_(?:R)?\d+)_\d+_kmer_size')
result_long
#result_long.to_csv('result_long_test.csv',sep='\t')

,column,OTU_count_nonzero,sample_name
0,V1_V9_1_3_kmer_size,9,V1_V9_1
1,V1_V9_1_4_kmer_size,15,V1_V9_1
2,V1_V9_1_5_kmer_size,14,V1_V9_1
3,V1_V9_1_6_kmer_size,11,V1_V9_1
4,V1_V9_1_7_kmer_size,11,V1_V9_1
5,V1_V9_2_3_kmer_size,9,V1_V9_2
6,V1_V9_2_4_kmer_size,12,V1_V9_2
7,V1_V9_2_5_kmer_size,13,V1_V9_2
8,V1_V9_2_6_kmer_size,10,V1_V9_2
9,V1_V9_2_7_kmer_size,11,V1_V9_2


In [90]:
result_long['kmer'] = result_long['column'].str.extract(r'V1_V9_(?:R)?\d+_(\d+)_kmer_size')
result_long.drop('column', axis=1, inplace=True)
result_long.fillna(0)
result_long['kmer'] = result_long['kmer'].astype('int')
result_long

,OTU_count_nonzero,sample_name,kmer
0,9,V1_V9_1,3
1,15,V1_V9_1,4
2,14,V1_V9_1,5
3,11,V1_V9_1,6
4,11,V1_V9_1,7
5,9,V1_V9_2,3
6,12,V1_V9_2,4
7,13,V1_V9_2,5
8,10,V1_V9_2,6
9,11,V1_V9_2,7


In [91]:
result_long=result_long.sort_values('kmer',ascending=True)

In [92]:
result_long

result_long.to_csv('kmers_3_7_V1_V9_OTU_count_long_table.csv', sep='\t', index=False)

**PLOTTING FROM RESULT_LONG DF( OPTIMAL)**

In [ ]:
fig, ax = plt.subplots()
for sample in result_long['sample_name'].unique():
    subset = result_long[result_long['sample_name'] == sample]
    ax.plot(subset['number_of_reads'], subset['OTU_count_nonzero'], marker='o', linestyle='-', label=sample)

ax.set_xlabel('Number of reads')
ax.set_ylabel('Count')
ax.set_title('OTU')
ax.set_ylim(-1, 35)
ax.set_xscale('log')
#ax.xaxis.set_ticks(result_melted["number_of_reads"])
ax.legend()
plt.show()
#plt.savefig('OTU_V3_V4.png')

In [ ]:
samples = pd.Series([col.rsplit('_', 2)[0] for col in result.columns]).drop_duplicates().tolist()
samples

In [ ]:
reads = pd.Series([col.split('_')[-2] for col in result.columns]).drop_duplicates().tolist()
reads.sort()
reads

TESTING RESHAPING DF ON DUMMY DATA

In [ ]:
# Assuming your initial dataframe is named df
data = {
    'V3_V4_1_100_reads': [10],
    'V3_V4_2_100_reads': [15],
    'V3_V4_3_100_reads': [20],
    'V3_V4_1_500_reads': [30],
    'V3_V4_2_500_reads': [30],
    'V3_V4_3_500_reads': [35]
}

df = pd.DataFrame(data)

# Melt the dataframe to go from wide to long format
df_long = df.melt(var_name='column', value_name='OTU_count_nonzero')

In [ ]:
df

In [ ]:
df_long

In [ ]:
# Extract sample names and number of reads
df_long['sample_name'] = df_long['column'].str.extract(r'(V3_V4_\d+)_\d+_reads')
df_long

In [ ]:
df_long['number_of_reads'] = df_long['column'].str.extract(r'V3_V4_\d+_(\d+)_reads')
df_long

In [ ]:
# Drop the original column names as they are no longer needed
df_long.drop('column', axis=1, inplace=True)
df_long

In [ ]:
# Pivot the table to get the desired format
df_pivot = df_long.pivot(index='number_of_reads', columns='sample_name', values='OTU_count_nonzero')
df_pivot

In [ ]:
# Reset the index to get number_of_reads as a column
df_final = df_pivot.reset_index()
df_final

In [ ]:
# Optionally, sort by number_of_reads if needed
df_final.columns

In [ ]:
df_final.columns.name = None
# Print the final dataframe
print(df_final)

PLOTS

In [ ]:
Color_collection = {}

for i in data_tax_df.index:
    
    Color_collection[i] = get_color(Color_collection)

In [ ]:
import plotly.express as px
import plotly.subplots as sp

fig = px.bar(data_tax_df.T, 
             x=data_tax_df.columns, 
             y=data_tax_df.index,
             width=1500, 
             height=900, 
            # color=data_tax_df.index,
             labels={'value': 'Relative abundance', 'index':'Samples'}, 
             template='simple_white',
             color_discrete_map=Color_collection)
fig.update_layout(yaxis_range=[0, 1], legend_title_text='Taxon', legend_title_side='top center')
fig.update_traces(marker_line_width=1.1, marker_line_color='#202020', opacity=0.8)
fig.update_yaxes(ticksuffix = "  ")
fig.update_xaxes(range=[-1, len(mearged_otu_table.T)+0.2], autorangeoptions_clipmax=len(data_tax_df.T))

#fig.update_layout(showlegend=False)
#os.mkdir("VIZ")
fig.write_image(f"VIZ/16S_{taxonomy_level}.pdf")
fig.write_image(f"VIZ/16S_{taxonomy_level}.png", scale=5)
fig.show()